In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#read label file csv
df = pd.read_csv(r'drones-for-ducks-classifications.csv')

#remove unused columns
df = df.iloc[: , 10:-1]

#remove special characters
df = df.replace('\{','', regex=True)
df = df.replace('\}','', regex=True)
df = df.replace('\"','', regex=True)
df = df.replace('\]','', regex=True)
df = df.replace('\[','', regex=True)
           
df['object_count'] = 0
df = df.replace('value:','value,', regex=True)

df.head()

/Users/rowanconverse/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,metadata,annotations,subject_data,object_count
0,"source:api,session:15e8c6c801b72d6bfd7dc7d33b2...","task:T0,task_label:Do you see birds in this im...","54652402:retired:null,Filename:BDA_12C_2018112...",0
1,"source:api,session:3ff627d3c29c1bb0f36bb9dcfb1...","task:T0,task_label:Do you see birds in this im...","54652421:retired:null,Filename:BDA_12C_2018112...",0
2,"source:api,session:6b97d1dd59d7f200bbcd1579493...","task:T1,task_label:Draw a rectangle around eac...","54652440:retired:null,Filename:BDA_18A4_201811...",0
3,"source:api,session:6a70be793547652e5bef12bead2...","task:T1,task_label:Draw a rectangle around eac...","54652428:retired:null,Filename:BDA_18A4_201811...",0
4,"source:api,session:6a70be793547652e5bef12bead2...","task:T1,task_label:Draw a rectangle around eac...","54652440:retired:null,Filename:BDA_18A4_201811...",0


In [3]:
len(df)

219254

In [4]:
for i in range(len(df)):
  df['object_count'][i] = str(df['annotations'][i]).count('x:')

df = df.dropna()

len(df)

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_8212/2952231697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['object_count'][i] = str(df['annotations'][i]).count('x:')


219251

In [6]:
ndf = df.query('object_count != 0')
ndf.head()

,metadata,annotations,subject_data,object_count
0,"source:api,session:15e8c6c801b72d6bfd7dc7d33b2...","task:T0,task_label:Do you see birds in this im...","54652402:retired:null,Filename:BDA_12C_2018112...",15
1,"source:api,session:3ff627d3c29c1bb0f36bb9dcfb1...","task:T0,task_label:Do you see birds in this im...","54652421:retired:null,Filename:BDA_12C_2018112...",3
3,"source:api,session:6a70be793547652e5bef12bead2...","task:T1,task_label:Draw a rectangle around eac...","54652428:retired:null,Filename:BDA_18A4_201811...",44
4,"source:api,session:6a70be793547652e5bef12bead2...","task:T1,task_label:Draw a rectangle around eac...","54652440:retired:null,Filename:BDA_18A4_201811...",44
5,"source:api,session:6a70be793547652e5bef12bead2...","task:T1,task_label:Draw a rectangle around eac...","54652421:retired:null,Filename:BDA_12C_2018112...",51


In [7]:
#write lines here to remove "metadata / subject dimensions / null data"
new = ndf.head(60)
#new.metadata
new = new.drop(new[new.iloc[i][0].split('naturalWidth:')[-1].split(',')[0].isdigit()==False].index)
#for i in range(len(new)):
#    if new.iloc[i][0].split('naturalWidth:')[-1].split(',')[0].isdigit() == False:
#        new.drop(new.index[i], inplace=True )
len(new)

IndexError: single positional indexer is out-of-bounds

In [8]:
#new = df.head(60)
#new.iloc[56][0].split('naturalWidth:')[-1].split(',')[0]
new.iloc[49,0]
#figured out the error-- in the blob containing the subject dimensions info, the line throwing an error has them as "null".
#Need to write a line that will discard a line with null data in this area in a prior step (see above)

'source:api,session:f7969bbcbaafcfa87a511ca51d0306a3f742c77c38e453560e81dd58b8bf207c,viewport:width:1920,height:935,started_at:2021-03-30T12:18:38.735Z,user_agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.105 Safari/537.36,utc_offset:-3600,finished_at:2021-03-30T12:18:41.462Z,live_project:true,interventions:opt_in:false,messageShown:false,user_language:en,subject_dimensions:null,subject_selection_state:retired:false,selected_at:2021-03-30T12:00:51.087Z,already_seen:false,selection_state:normal,finished_workflow:false,user_has_finished_workflow:false,workflow_translation_id:41322'

In [18]:
sublabels = pd.DataFrame()

for i in range(len(new)):
  imW = new.iloc[i][0].split('naturalWidth:')[-1].split(',')[0]
  imH = new.iloc[i][0].split('naturalHeight:')[-1].split(',')[0]
  file_name = new.iloc[i][2].split('Filename:')[-1].split('.')[0]
  annotation = new.iloc[i][1].split('value')[-2].split('x:')
  annotation.pop(0)

  for j in range(len(annotation)):
    x = (float(annotation[j].split(',')[0]) + (float(annotation[j].split('width:')[-1].split(',')[0])/2))/float(imW)
    y = (float(annotation[j].split('y:')[-1].split(',')[0]) + (float(annotation[j].split('height:')[-1].split(',')[0])/2))/float(imH)
    w = float(annotation[j].split('width:')[-1].split(',')[0])/float(imW)
    h = float(annotation[j].split('height:')[-1].split(',')[0])/float(imH)
    label = annotation[j].split('tool_label:')[-1].split(',')[0]
    row = {'FileName': file_name , 'X': x  , 'Y': y , 'Width': w , 'Height': h , 'Label': label}
    sublabels = sublabels.append(row,ignore_index = True)
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [19]:
sublabels.head()

,FileName,X,Y,Width,Height,Label
0,BDA_18A4_20181106_1 copy,0.460567,0.393007,0.031016,0.036001,Duck
1,BDA_18A4_20181106_1 copy,0.532713,0.248186,0.037759,0.031091,Duck
2,BDA_18A4_20181106_1 copy,0.433596,0.262096,0.036410,0.022910,Duck
3,BDA_18A4_20181106_1 copy,0.184792,0.290733,0.037759,0.034364,Duck
4,BDA_18A4_20181106_1 copy,0.256264,0.109911,0.032365,0.036001,Duck


In [13]:
sublabels = sublabels.reindex(['FileName','Label','X','Y','Width','Height'], axis=1)
sublabels.head()

,FileName,Label


In [ ]:
#cleaning label categories; some species-specific data is mixed in from a beta test 
di = {'Canada Goose (*Branta canadensis*)': 'Goose', 'Mallard Duck (*Anas platyrhynchos*)': 'Duck', 'Sandhill Crane (*Grus canadensis*)': 'Crane', 'Blue-Winged Teal (*Anas discors*)': 'Duck', 'American Widgeon (*Mareca americana*)': 'Duck','Northern Pintail (*Anas acuta*)': 'Duck', 'Northern Shoveler (*Spatula clypeata*)': "Duck", 'Kittiwakes': 'Duck', 'Guillemots': 'Duck'}
labels = labels.replace(di)

In [ ]:
#Original image name from tile name
labels["Original"] = labels['FileName'].str[:-5]
labels["Original"] = labels["Original"].str.rstrip("_")

In [ ]:
#Management units from image names
for i in range(len(labels)):
    labels["Unit"][i] = labels.iloc[i][2].split("_")[1].lower()

In [ ]:
#Empty images
empty = df.loc[df['object_count'] == 0]
zeros = pd.DataFrame()
for i in range(len(df)):
    file_name = empty.iloc[i][2].split('Filename:')[-1].split('.')[0]
    for j in range(len(annotation)):
         row = {'FileName': file_name}
    zeros = zeros.append(row,ignore_index = True)
print(i)    

In [ ]:
print('High-level statistics:\n')

print('Original Images: ' +str(len(labels.Original.unique())))
print('Image Tiles: '+str(len(labels.FileName.unique())))
print('Categories: '+str(len(labels["Label"].unique())) + ", " + str(labels["Label"].unique()))
print('Zooniverse Subject Annotations: '+str(len(df.annotations)))
print('Empty Image Tiles: ' + str(len(zeros.FileName.unique())))
print('Unique Bird Labels: ' + str(len(labels)))
print('Locations: ' + str(len(labels["Unit"].unique())))

In [ ]:
#Images by category
catgroup = labels.groupby("Label")["FileName"].unique().to_frame().reset_index()
catgroup['Images'] = catgroup['FileName'].str.len()
catgroup = catgroup.sort_values('Images', ascending=False)
ax = catgroup.plot(x = "Label", y = "Images", kind = "bar", xlabel = "Categories", ylabel = "Number of Images", title = "Images by Category", legend = False)

In [ ]:
#Images by location
group = labels.groupby("FileName")["Unit"].unique().to_frame().reset_index()
group["Unit"] = labels["Unit"].str.rstrip("[]")
group["Unit"].value_counts().plot(kind='bar', xlabel = "Managment Unit", ylabel = "Number of Images", title = "Images by Location")

In [ ]:
#Categories by location
pd.crosstab(labels['Unit'],labels['Label']).plot.bar(ylabel = "Individual Bird Labels", title = "Number of Annotations per Category by Management Unit")